In [1]:
import csv
import math
import time
import warnings
import operator
import statistics
import seaborn as sns
import pandas as pd
import numpy as np
import geopy.distance
import matplotlib.pyplot as plt

from dateutil.parser import parse
from datetime import datetime
from scipy import stats
from IPython.display import display_html
from multiprocessing import Pool
from sklearn.model_selection import train_test_split

plt.style.use('fivethirtyeight')
warnings.filterwarnings('ignore')
%matplotlib inline

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.float_format = '{:.2f}'.format

In [2]:
df_ads = pd.read_csv('gs://aes-datahub-0002-curated/Outage_Restoration/Historical_Data/Master_Dataset/OMS_IPL_V10.4_IQR_FULL.csv')
df_ads.head()

,OUTAGE_ID,INCIDENT_ID,STRCTUR_NO,EVENT,CREATION_DATETIME,ENERGIZED_DATETIME,CIRCT_ID,DNI_EQUIP_TYPE,SUBST_ID,CALL_QTY,DOWNSTREAM_CUST_QTY,KEY_CUST_QTY,ETR_DATETIME,CUST_QTY,DAY_FLAG,TTR,POLE_CLUE_FLG,PART_LIGHT_CLUE_FLG,EMERGENCY_CLUE_FLG,POWER_OUT_CLUE_FLG,OPEN_DEVICE_CLUE_FLG,TREE_CLUE_FLG,WIRE_DOWN_CLUE_FLG,IVR_CLUE_FLG,EQUIPMENT_CLUE_FLG,TRANSFORMER_CLUE_FLG,OH_CAUSE_FLG,UG_CAUSE_FLG,ANIMAL_CAUSE_FLG,WEATHER_CAUSE_FLG,WEATHER_COLD_CAUSE_FLG,PUBLIC_CAUSE_FLG,WEATHER_LIGHTNING_CAUSE_FLG,WEATHER__SNOW_CAUSE_FLG,WEATHER__WIND_CAUSE_FLG,WEATHER__HEAT_CAUSE_FLG,CUST_REQUEST_CAUSE_FLG,WEATHER__FLOOD_CAUSE_FLG,STREET_CAUSE_FLG,SUBSTATION_CAUSE_FLG,TREE_CAUSE_FLG,MISCELLANEOUS_CAUSE_FLG,NO_CAUSE_FLG,PLANNED_CAUSE_FLG,NO_OUTAGE_CAUSE_FLG,FUSE_OCCURN_FLG,CUST_EQUIP_OCCURN_FLG,POLE_OCCURN_FLG,TRANSFORMER_OCCURN_FLG,METER_OCCURN_FLG,SERVICE_OCCURN_FLG,CABLE_OCCURN_FLG,ST_OCCURN_FLG,FIRE_OCCURN_FLG,FOUND_OPEN_OCCURN_FLG,PUBLIC_SAFETY_OCCURN_FLG,WIRE_OCCURN_FLG,SWITCH_OCCURN_FLG,REGULATOR_OCCURN_FLG,CUTOUT_OCCURN_FLG,CAP_BANK_OCCURN_FLG,RECLOSER_OCCURN_FLG,OH_OCCURN_FLG,PRIORITY_VAL_1.0,PRIORITY_VAL_2.0,PRIORITY_VAL_3.0,PRIORITY_VAL_5.0,CITY_NAM,LAT,LONG,ZONE,NO_OF_POWER_OUT_CLUE_PER_DAY,NO_OF_OPEN_DEVICE_CLUE_PER_DAY,NO_OF_IVR_CLUE_PER_DAY,NO_OF_ANIMAL_CAUSE_PER_DAY,NO_OF_WIRE_OCCURN_PER_DAY,Marker_Location,Hour_Sin,Hour_Cos,RANK_SUBSEQUENT_OUTAGES,LIVE_OUTAGE,Outages_in_last_1hr,Outages_in_last_2hr,Outages_in_last_3hr,Outages_in_last_4hr,Outages_in_last_5hr,Outages_in_last_6hr,Outages_in_last_7hr,Outages_in_last_8hr,Outages_in_last_9hr,Outages_in_last_10hr,Weekday,Weekend_flag,Live_outage_group,Priority_Customer_Qty,Dis_From_Live_Centriod_div_Cust_qty,Priority_Dist_Customer_Qty,Dispatch_Location,cldCvrAvg,cldCvrMax,cldCvrMin,dewPtAvg,dewPtMax,dewPtMin,feelsLikeAvg,feelsLikeMax,feelsLikeMin,heatIndexAvg,heatIndexMax,heatIndexMin,mslPresAvg,mslPresMax,mslPresMin,precip,radSolarAvg,radSolarMax,radSolarTot,relHumAvg,relHumMax,relHumMin,sfcPresAvg,sfcPresMax,sfcPresMin,snowfall,spcHumAvg,spcHumMax,spcHumMin,tempAvg,tempMax,tempMin,wetBulbAvg,wetBulbMax,wetBulbMin,windChillAvg,windChillMax,windChillMin,windDir100mAvg,windDir80mAvg,windDirAvg,windSpd100mAvg,windSpd100mMax,windSpd100mMin,windSpd80mAvg,windSpd80mMax,windSpd80mMin,windSpdAvg,windSpdMax,windSpdMin,tempRange,windSpdRange,sfcPresRange,cldCvrRange,relHumRange,relHumRatio,sfcPresRatio,Active_no_of_trucks_per_day,Month,Year,Period,Date
0,2001309371,2000383861.00,3059,BLUE SKY,2007-01-01 03:33:00,2007-01-01 03:48:00,3059.00,SSUB,305.00,4,177.00,1.00,2007-01-01 05:03:00,177.00,False,15.00,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,True,False,False,False,INDIANAPOLIS,39.88,-86.11,ZONE3,7,1,5,2,1,Marker4,0.71,0.71,1.00,0,0,0,0,0,0,0,0,0,0,0,Monday,False,1,1.00,0.00,1.00,ARL.,92.00,100.00,44.00,33.90,39.50,30.00,30.30,36.70,24.30,38.50,43.70,34.50,1016.40,1026.00,1008.90,0.00,28.50,125.40,684.30,83.30,90.30,79.30,986.80,995.80,980.10,0.00,4.20,5.20,3.60,38.50,43.70,34.50,36.60,41.70,32.80,30.30,36.70,24.30,269.00,269.00,267.00,16.80,20.30,8.80,17.00,20.60,8.90,14.40,17.80,6.70,9.20,11.10,15.70,56.00,11.00,1.14,1.02,nan,1,2007,2007-1,2007-01-01 00:00:00
1,2001309379,2000383864.00,494-B/403,BLUE SKY,2007-01-01 08:16:14,2007-01-01 10:04:24,1252.00,FUSE,125.00,1,5.00,0.00,2007-01-01 11:46:14,5.00,True,108.17,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,INDIANAPOLIS,39.78,-86.27,ZONE4,7,1,5,2,1,Marker10,0.87,-0.50,2.00,0,0,0,0,0,1,1,1,1,1,1,Monday,False,2,2.00,1.05,2.00,MILL,93.00,100.00,43.00,33.20,39.10,29.50

In [3]:
%%time
df_facility_job_his = pd.read_csv('gs://aes-datahub-0001-raw/OMS/2002-2020/IPL/HIS_FACILITY_JOB_IPL.csv', sep = ",", encoding = "ISO-8859-1")

CPU times: user 27.3 s, sys: 3.32 s, total: 30.6 s
Wall time: 57.6 s


In [4]:
print(list(df_facility_job_his.columns))

['FAC_JOB_ID', 'CIRCT_ID', 'MAJ_OTG_ID', 'EQUIP_STN_NO', 'DIST_NO', 'HOST_SEQ_ID', 'PRIORITY_VAL', 'CUST_QTY', 'CLUE_CD', 'CLUE_DESC', 'CREATION_DATETIME', 'CALL_QTY', 'KEY_CUST_QTY', 'SPLIT_FAC_JOB_FLG', 'CAUSE_CD', 'CAUSE_DESC', 'OCCURN_CD', 'OCCURN_DESC', 'CLIMATIC_CD', 'CLIMATIC_DESC', 'CITY_NAM', 'LOC_DESC', 'WRK_ORD_NUM', 'COMMENT_TEXT', 'CALL_ID', 'KVA_VAL', 'BOOK_NO', 'ADDRESS', 'CIRCT_NAM', 'CLUE_CD2', 'INSERTED_DATE', 'DOWNSTREAM_KVA_VAL', 'DOWNSTREAM_CUST_QTY', 'COMPL_DATETIME', 'TOT_LOSS_POWER_FLG', 'ISOLATED_TO_CUST_FLG', 'PLANNED_OUTAGE_FLG', 'ROUTINE_FLG', 'DNI_EQUIP_TYPE', 'SUBST_ID', 'WORK_ORD_1_NO', 'WORK_ORD_2_NO', 'WORK_ORD_3_NO', 'WORK_ORD_4_NO', 'WORK_ORD_5_NO', 'ENERGIZED_DATETIME', 'DISPLAY_TEXT', 'POLICE_OPERATOR_ID', 'POLICE_INC_NO', 'FIRE_OPERATOR_ID', 'FIRE_INC_NO', 'CAD_ID', 'STRCTUR_NO', 'FAC_JOB_PARENT_ID', 'MAJ_INCIDENT_FLG', 'MAJ_INCIDENT_CAUSE', 'ZONE_DESC', 'DIST_DESC', 'ZONE_ID', 'GEO_DIST_NO', 'ETR_DATETIME', 'SUBST_SHUTDOWN_FLG', 'HIS_FAC_JOB_COMME

In [5]:
df_facility_job_his_new = df_facility_job_his[['FAC_JOB_ID', 'CAD_ID']]
del df_facility_job_his

In [6]:
df_merged = pd.merge(df_ads, df_facility_job_his_new, how='left', left_on=['OUTAGE_ID'], right_on=['FAC_JOB_ID'])
print(df_merged.CAD_ID.isnull().sum())

16327


In [7]:
cad_isr = pd.read_csv('gs://aes-datahub-0001-landing/CAD/IPL/CAD.ISR.csv',sep = ";", encoding = "ISO-8859-1")
cad_his_isr = pd.read_csv('gs://aes-datahub-0001-landing/CAD/IPL/CAD.HIS_ISR.csv', sep = ";", encoding = "ISO-8859-1")

In [8]:
cad_final = cad_isr.append(cad_his_isr)

In [9]:
del cad_isr, cad_his_isr

In [10]:
print(list(cad_final.columns))

['ISR_NO', 'ASSIGN', 'AGENCY_CODE', 'DISPATCH_LEVEL', 'CIVIC_NO', 'STREET_NAME1', 'STREET_NAME2', 'P_ARTICLE1', 'P_ARTICLE2', 'P_DIR1', 'P_DIR2', 'P_STREET_TYPE1', 'P_STREET_TYPE2', 'PHONE_NO', 'ZIP_CODE', 'HAZMAT', 'HANDICAP', 'GEOBASE_COMMENT120', 'GEOBASE_SOP', 'APPT_NO', 'ALPHA_CIVIC', 'DISTRICT_NAME', 'CITY_ABBR', 'X_STREET1', 'X_STREET2', 'P_LOC_ENVIRONMENT', 'GEOBASE_ID', 'LOCATION_NAME', 'P_LOC_TYPE', 'P_PROXIMITY', 'LEVEL5', 'COMPL_ADDRESS', 'COMPL_CITY_ABBR', 'COMPL_TEL', 'P_COMPL_TEL_EXT', 'COMPL_NAME', 'COMPL_ADDINFO', 'COMPL_CONTACT', 'TAG_FLAG', 'INIT_SERVICE_CODE', 'FINAL_SERVICE_CODE', 'P_ALARM_LEVEL', 'P_ORIGIN', 'PRIORITY', 'P_STATUS', 'STATUS_DATETIME', 'S_STATUS_NOTE', 'LONGITUDE', 'LATITUDE', 'CENTROID', 'GEOBASE_VALIDATION', 'GEOBASE_TYPE', 'EXT_SERVICES', 'COMMERCIAL_LOCAL', 'S_MODE', 'PERSON', 'EQUIPMENT', 'RELATION', 'LOG', 'LOCK_DATETIME', 'SCHEDULE_DATETIME', 'DEFERRED_DATETIME', 'IS_AN_APPOINTMENT', 'P_ISR_GROUPING', 'MOD_LOC', 'MOD_SERV_CODE', 'MOD_PRIORITY

In [11]:
cad_final_v1 = cad_final[['ISR_NO', 'HANDLING_UNIT', 'UNIT_COUNT']]
del cad_final

In [13]:
df_final = pd.merge(df_merged, cad_final_v1, left_on=['CAD_ID'], right_on=['ISR_NO'])

In [15]:
print(df_merged.shape)
print(df_final.shape)

(76776, 162)
(60449, 165)


In [16]:
df_final.to_csv("gs://aes-datahub-0002-curated/Outage_Restoration/Historical_Data/Master_Dataset/OMS_IPL_OUTAGE_TRUCKS_10212020.csv",index=False)